In [10]:
import pandas as pd
import numpy as np
import collections


In [2]:
def return_column_number(df, column_name):
    columns = df.columns
    for i in range(len(columns)):
        if columns[i] == column_name:
            return i
        
    print("No columns with this name")

In [3]:
df = pd.read_csv("B:\Downloads\Road_Traffic_Fine_Management_Process.xes\Road_Traffic_Fine_Management_Proc-training.csv", parse_dates=[4])
df = df.sort_values(by=["case concept:name", "event time:timestamp"])
df = df.reset_index()
current_trace = ""
event_in_trace = 0
df['event_in_trace'] = 0
case_concept_column_number = return_column_number(df, "case concept:name")
event_in_trace_column_number = return_column_number(df, "event_in_trace")

"""
Creates a new columns that tells in which position the event is in its own trace
"""
for i in range(len(df)):
    if df.iat[i, case_concept_column_number] == current_trace:
        event_in_trace += 1
        df.iat[i, event_in_trace_column_number] = event_in_trace
    else:
        event_in_trace = 0
        current_trace = df.iat[i, case_concept_column_number]
        df.iat[i, event_in_trace_column_number] = event_in_trace
        
        

In [4]:
df.head()

,index,eventID,case concept:name,event concept:name,event lifecycle:transition,event time:timestamp,event_in_trace
0,206285,1,A1,Send Fine,complete,2006-05-12,0
1,188244,0,A1,Create Fine,complete,2006-07-24,1
2,189251,4294967296,A100,Create Fine,complete,2006-02-08,0
3,207056,4294967297,A100,Send Fine,complete,2006-12-12,1
4,213793,4294967298,A100,Insert Fine Notification,complete,2007-01-15,2


In [22]:
"""
Calculate  how long the longest trace
"""
df["time_difference"] = pd.Timedelta(0, 'd')
df["predicted_date_next_event"] = pd.Timedelta(0, 'd')
df["predicted_name_next_event"] = ""
case_concept_column_number = return_column_number(df, "case concept:name")
event_concept_column_number = return_column_number(df, "event concept:name")
event_in_trace_column_number = return_column_number(df, "event_in_trace")
event_time_column_number = return_column_number(df, "event time:timestamp")
predicted_date_column_number = return_column_number(df, "predicted_date_next_event")
predited_name_column_number = return_column_number(df, "predicted_name_next_event")
time_difference_column_number = return_column_number(df, "time_difference")
longest_trace = df["case concept:name"].value_counts()[0]
all_time_differences = {x:(0, pd.Timedelta(0,'d'))for x in range(longest_trace)}
average_difference = {}
all_events = {x:[] for x in range(longest_trace)}
for i in range(len(df)):
    
    
    if (i < len(df)-1):
        all_events[df.iat[i, event_in_trace_column_number]].append(df.iat[i+1, event_concept_column_number])
        
        if (df.iat[i+1, case_concept_column_number] == df.iat[i, case_concept_column_number]):
            df.iat[i, time_difference_column_number] = df.iat[i+1, event_time_column_number] - df.iat[i, event_time_column_number]
        else:
            df.iat[i, time_difference_column_number] = "last event in trace"
            
for i in range(longest_trace):
    df_per_i = df[df["event_in_trace"] == i]
    average_difference[i] = 0
    df_per_i = df_per_i[df_per_i["time_difference"] != "last event in trace"]
    if len(df_per_i == 0):
        total  = 0
        for j in range(len(df_per_i)):
            total = df_per_i.iat[j, time_difference_column_number].days + total
        average_difference[i] = int(total/len(df_per_i))

for i in range(len(all_events)):
    all_events[i] = collections.Counter(all_events[i]).most_common()[0][0]
    
for i in range(len(df)):
    df.iat[i, predicted_date_column_number] = df.iat[i,event_time_column_number] + pd.Timedelta(average_difference[df.iat[i, event_in_trace_column_number]], "d")
    df.iat[i, predited_name_column_number] = all_events[df.iat[i, event_in_trace_column_number]]


In [25]:
df[df["event_in_trace"] == 19].head(50)

,index,eventID,case concept:name,event concept:name,event lifecycle:transition,event time:timestamp,event_in_trace,time_difference,predicted_date_next_event,predicted_name_next_event
96820,380115,116947664502803,C20817,Payment,complete,2011-09-04,19,last event in trace,2011-09-04 00:00:00,Insert Fine Notification


In [11]:
for i in range(len(all_events)):
    all_events[i] = collections.Counter(all_events[i])

In [21]:
print(all_events[0].most_common()[0][0])

Send Fine


In [14]:
unique_events = df["case concept:name"].unique()
print(type(unique_events[0]))

<class 'str'>
